In [8]:
import xgboost as xgb
import pandas as pd
import numpy as np
from datetime import timedelta
import dataframe_image as dfi
import matplotlib.pyplot as plt
import sklearn.metrics as mtr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle

pd.options.display.max_colwidth = 200

In [9]:
df = pd.read_csv("../Bases/Cleaned/rota33642_weather.csv")

df["data_partida"] = pd.to_datetime(df["data_partida"])

df = df.sort_values(by=["data_partida"], ascending=True)

df["tempo_viagem_1"] = df["tempo_viagem"]
df["tempo_viagem_2"] = df["tempo_viagem"]
df["tempo_viagem_3"] = df["tempo_viagem"]
df["tempo_viagem_4"] = df["tempo_viagem"]

df["tempo_viagem_1"] = df["tempo_viagem_1"].shift(1)
df["tempo_viagem_2"] = df["tempo_viagem_2"].shift(2)
df["tempo_viagem_3"] = df["tempo_viagem_3"].shift(3)
df["tempo_viagem_4"] = df["tempo_viagem_4"].shift(4)

df.to_csv('../Bases/Cleaned/rota33642_bf_times_F3.csv',index=False)
df.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,qtdDiasAno,tipo_dia,turno_dia,hora_dia,Precipitacao,Umidade,Vento,Temperatura,Descricao Chuva,Calor,tempo_viagem_1,tempo_viagem_2,tempo_viagem_3,tempo_viagem_4
9313,2019-07-01 04:24:03,703,1,1561965843,182,1,1,4,0.0,62,0.3,19.35,0,1,NaN,NaN,NaN,NaN
9312,2019-07-01 04:49:57,801,1,1561967397,182,1,1,4,0.0,66,0.1,18.80,0,1,703.0,NaN,NaN,NaN
9311,2019-07-01 04:58:33,781,1,1561967913,182,1,1,4,0.0,66,0.1,18.80,0,1,801.0,703.0,NaN,NaN
9310,2019-07-01 05:07:10,781,1,1561968430,182,1,2,5,0.0,66,0.1,18.80,0,1,781.0,801.0,703.0,NaN
9309,2019-07-01 05:16:42,787,1,1561969002,182,1,2,5,0.0,66,0.1,18.80,0,1,781.0,781.0,801.0,703.0


In [10]:
best_models = None
best_metric = None
best_cenario = [0,0,0,0]

In [11]:
data_atual = df["data_partida"].dt.date.min() + timedelta(days=42)



df_teste = df[df["data_partida"].dt.date > data_atual]
df_treino = df[df["data_partida"].dt.date <= data_atual]

for cenario in range(4):
        if cenario == 0:
                df_teste = df_teste.iloc[4:]
                df_treino = df_treino.iloc[4:]
                chuva = 'C'
                colT = 'T'

                x_teste = df_teste[["dia_semana", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4", "Descricao Chuva"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["dia_semana", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4", "Descricao Chuva"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 1:
                df_teste = df_teste.iloc[4:]
                df_treino = df_treino.iloc[4:]
                chuva = 'S'
                colT = 'T'

                x_teste = df_teste[["dia_semana", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["dia_semana", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4"]]
                y_treino = df_treino["tempo_viagem"]
        
        elif cenario == 2:
                df_teste = df_teste.iloc[4:]
                df_treino = df_treino.iloc[4:]
                chuva = 'C'
                colT = 'S'                

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4", "Descricao Chuva"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4", "Descricao Chuva"]]
                y_treino = df_treino["tempo_viagem"]
        
        else:
                df_teste = df_teste.iloc[4:]
                df_treino = df_treino.iloc[4:]
                chuva = 'S'
                colT = 'S' 

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4"]]
                y_treino = df_treino["tempo_viagem"]


        # Hyperparâmetros do XGBoosting
        params_xgb = {
                "n_estimators": list(range(100, 1100, 100)), #Number of gradient boosted trees. Equivalent to number of boosting rounds
                "max_depth": list(range(2, 15)),#Maximum tree depth for base learners.
                "min_child_weight": list(range(1, 11)),#Minimum sum of instance weight(hessian) needed in a child.
                "learning_rate": [0.3, 0.2, 0.1, 0.05, 0.01, 0.005],#Boosting learning rate (xgb’s “eta”)
                "gamma": np.arange(0, 0.7, 0.1)#Minimum loss reduction required to make a further partition on a leaf node of the tree.
                }

        # Hyperparâmetros do Random Forest
        params_rf = {
                "n_estimators": list(range(100, 1100, 100)),#The number of trees in the forest.
                "bootstrap": [True, False],#Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.
                "max_depth": list(range(2, 15)),#The maximum depth of the tree.
                "max_features": [1.0, "sqrt", "log2"],#The number of features to consider when looking for the best split:
                "min_samples_leaf": list(range(1, 11)),#The minimum number of samples required to split an internal node
                "min_samples_split": list(range(2, 11)),#The minimum number of samples required to be at a leaf node.
                }

        # Hyperparâmetros do SVR
        params_svr = {
                "C": [0.1, 1, 10, 100, 1000], #Regularization parameter. The strength of the regularization is inversely proportional to C
                "gamma": ["scale", "auto"],#Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
                }

        #Linear Regression não tem hyper parametrização


        #Criação e fit do modelo XGB
        modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        xgb_rand_search.fit(x_treino, y_treino)
        modelo_xgb = xgb_rand_search.best_estimator_

        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)])


        #Criação e fit do modelo Random Forest
        modelo_rf = RandomForestRegressor()
        rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        rf_rand_search.fit(x_treino, y_treino)
        modelo_rf = rf_rand_search.best_estimator_

        modelo_rf.fit(x_treino, y_treino)


        #Criação e fit do modelo SVR
        modelo_svr = SVR(kernel = 'rbf')
        svr_rand_search = RandomizedSearchCV(modelo_svr, params_svr, scoring="neg_mean_squared_error", n_iter=10, verbose=True, cv=10, n_jobs=-1, random_state=123)
        svr_rand_search.fit(x_treino, y_treino)
        modelo_svr = svr_rand_search.best_estimator_

        modelo_svr.fit(x_treino, y_treino)


        #Criação e fit do modelo Linear Regression
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)

        df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
        df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
        df_teste["predicao_rf"] = modelo_rf.predict(x_teste)
        df_teste["predicao_svr"] = modelo_svr.predict(x_teste)

        df_teste.to_csv(f"../Bases/Predictions/rota33642_predictions_F3_weather_{chuva}{colT}.csv",index=False)

        RMSE_xgb = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_xgb"], squared=False) 
        RMSE_rf = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_rf"], squared=False)       
        RMSE_svr = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_svr"], squared=False) 
        RMSE_lr = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_lr"], squared=False) 

        models = [modelo_xgb, modelo_rf, modelo_svr, modelo_lr]
        metrics = [RMSE_xgb,RMSE_rf,RMSE_svr,RMSE_lr]

        if best_models is None:
                best_models = models
                best_metrics = metrics
        else:
                for i in range(4):
                        if best_metrics[i] > metrics[i]:
                                best_models[i] = models[i]
                                best_metrics[i] = metrics[i]
                                best_cenario[i] = f'{chuva}{colT}'



Fitting 10 folds for each of 40 candidates, totalling 400 fits
[17:52:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[17:52:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1178.45399
[1]	validation_0-rmse:1172.70583
[2]	validation_0-rm

In [12]:
models_name = ['xgb', 'rf', 'svr', 'lr']
for i in range(4):
    with open (f'ModelsF3/rota33642_F3_weather_{models_name[i]}_cen_{best_cenario[i]}.pkl', 'wb') as m:
        pickle.dump(best_models[i], m)